In [1]:
from setup import *
from agents import *
from env import *

In [117]:
torch.set_default_dtype(torch.float32)
gameTypes, numBuyers, numSellers, numTokens, numRounds, numPeriods, numSteps, seed = '1001', 2, 2, 3, 1, 50000, 9, 42
disclosure = ['rnd', 'period', 'step','currentBid','currentAsk','buy','sell','price','sale']
buyerStrategies = ['PPO', 'PPO']
sellerStrategies = ['PPO', 'PPO']
gameData = [gameTypes, numBuyers, numSellers, numTokens, numRounds, numPeriods, numSteps, seed]
buyers, sellers = generateAgents(gameData,buyerStrategies,sellerStrategies,disclosure)
log = Log(gameData, buyerStrategies, sellerStrategies, disclosure)

for rnd in range(numRounds):
    roundData = roundSetup(*gameData)
    redemptionValues, tokenCosts, demand, supply, prices, peq, qeq  = roundData[0:7]
    [buyerReservationPrices, sellerReservationPrices, buyerSurplus, sellerSurplus, totalSurplus, buyerSurplusFrac, sellerSurplusFrac] = roundData[7:]
    log.addRound([rnd] + roundData)
    resetRounds(buyers, sellers, redemptionValues, tokenCosts)
    Rlist = []
    Rlist2 = []
    Rlist3 = []
    bids_made = []
    for period in range(numPeriods):
        resetPeriods(buyers, sellers)
        R = 0
        R2 = 0
        bidsmade = []
        for step in range(numSteps):
            resetSteps(buyers, sellers)
            bids, asks = collectOffers(buyers, sellers)
            currentAsk, currentAskIdx, currentBid, currentBidIdx = bestOffers(bids, asks)
            price, buy, sell = trade(buyers, sellers, currentAsk, currentAskIdx, currentBid, currentBidIdx)
            bprofit, sprofit = 0, 0
            bidsmade.append(bids[0])
            if price > 0:
                buyers[currentBidIdx].transact(price)
                sellers[currentAskIdx].transact(price)
                bprofit = buyers[currentBidIdx].stepProfits
                sprofit = sellers[currentAskIdx].stepProfits
            R += buyers[0].stepProfits
            R2 += buyers[1].stepProfits
            log.addStep([rnd, period, step, bids, asks, currentBid, currentBidIdx, currentAsk, currentAskIdx, buy, sell, price, price>0, bprofit, sprofit])
            observe(buyers, sellers, log.disclose()) # observe new state
            updateStates(buyers, sellers)
        Rlist.append(R)
        Rlist2.append(R2)
        Rlist3.append(R+R2)
        bids_made.append(np.nanmean(bidsmade))
        updatePolicy(buyers, sellers)
        print(period, np.round(np.mean(Rlist[-100:]),1),  np.round(np.mean(Rlist2[-100:]),1), np.round(np.mean(Rlist3[-100:]),1),np.round(np.nanmean(bids_made[-100:]),1) )
    #if period % (numPeriods/100) == 0:
    #    print(f'\n{period}')
        #display(log.getPeriod(rnd, period).tail(100)[['bprofit', 'currentBidIdx']].groupby('currentBidIdx').sum())
        #display(log.getPeriod(rnd, period).tail(100)[['sprofit', 'currentAskIdx']].groupby('currentAskIdx').sum())
        #display(log.getPeriod(rnd,period))

0 50.5 71.2 121.7 37.6
1 110.6 35.6 146.2 42.8
2 132.6 23.7 156.4 46.4
3 124.8 29.2 154.0 44.8
4 114.1 37.3 151.3 44.3
5 95.1 37.9 133.0 42.2
6 88.6 39.0 127.6 41.7
7 77.5 39.9 117.4 41.1
8 80.2 37.4 117.6 40.0
9 75.4 35.6 111.0 39.5
10 68.6 33.4 101.9 39.2
11 62.9 29.9 92.8 38.8
12 62.2 30.3 92.5 38.2
13 57.8 28.9 86.6 37.7
14 60.7 29.6 90.3 37.4
15 59.5 27.9 87.4 37.7
16 56.0 28.3 84.3 37.5
17 54.9 27.3 82.2 37.4
18 55.2 28.0 83.2 37.7
19 52.5 26.1 78.6 37.4
20 55.4 25.7 81.0 37.7
21 58.7 24.5 83.2 37.9
22 60.3 24.8 85.1 37.9
23 57.8 23.6 81.4 38.0
24 57.5 22.8 80.3 37.8
25 59.0 21.5 80.4 38.0
26 60.1 21.1 81.2 38.4
27 62.9 20.3 83.2 38.5
28 62.7 20.0 82.7 38.6
29 61.6 19.8 81.3 38.7
30 62.4 19.6 82.0 38.9
31 62.5 20.0 82.6 39.0
32 64.3 19.4 83.7 39.3
33 65.6 18.8 84.4 39.5
34 65.9 18.3 84.2 40.1
35 67.5 17.8 85.2 40.2
36 68.1 17.4 85.5 40.4
37 68.0 17.0 85.0 40.7
38 68.6 16.3 84.9 40.9
39 68.8 15.8 84.6 41.1
40 68.3 15.5 83.7 41.4
41 68.6 15.7 84.2 41.7
42 68.6 15.5 84.1 41.8
43 69.

KeyboardInterrupt: 

In [102]:
log.roundData.buyerValues.item(), log.roundData.sellerCosts.item()

(array([[100. ,  93. ,  49.1],
        [ 66.8,  66.2,  50.8]]),
 array([[ 1. , 51.3, 59.9],
        [34.6, 41.4, 75.1],
        [41.6, 50.4, 50.8],
        [ 0. , 35. , 43.3],
        [28.2, 54.1, 65.9],
        [ 2.8, 38.9, 42.1]]))

In [114]:
log.roundData[['peq', 'qeq', 'buyerSurplus', 'sellerSurplus', 'buyerSurplusFrac']]

,peq,qeq,buyerSurplus,sellerSurplus,buyerSurplusFrac
0,36.9,6,204.5,119.8,0.6


In [109]:
log.getPeriod(0,128)

,rnd,period,step,bids,asks,currentBid,currentBidIdx,currentAsk,currentAskIdx,buy,sell,price,sale,bprofit,sprofit
1152,0,128,0,"[73.6, 12.4]","[1.0, 34.6, 41.6, 0.0, 28.2, 2.8]",73.6,0.0,0.0,3,True,True,36.80,True,63.20,36.80
1153,0,128,1,"[73.7, 89.5]","[1.0, 34.6, 41.6, 35.0, 28.2, 2.8]",89.5,1.0,1.0,0,True,True,45.25,True,21.55,44.25
1154,0,128,2,"[74.4, 31.9]","[51.3, 34.6, 41.6, 35.0, 28.2, 2.8]",74.4,0.0,2.8,5,True,True,38.60,True,54.40,35.80
1155,0,128,3,"[60.5, 0.0]","[51.3, 34.6, 41.6, 35.0, 28.2, 38.9]",60.5,0.0,28.2,4,True,True,44.35,True,4.75,16.15
1156,0,128,4,"[nan, 0.0]","[51.3, 34.6, 41.6, 35.0, 54.1, 38.9]",0.0,1.0,34.6,1,True,False,34.60,True,31.60,0.00
1157,0,128,5,"[nan, 0.0]","[51.3, 41.4, 41.6, 35.0, 54.1, 38.9]",0.0,1.0,35.0,3,True,False,35.00,True,15.80,0.00
1158,0,128,6,"[nan, nan]","[51.3, 41.4, 41.6, 43.3, 54.1, 38.9]",NaN,NaN,38.9,5,NaN,NaN,NaN,False,0.00,0.00
1159,0,128,7,"[nan, nan]","[51.3, 41.4, 41.6, 43.3, 54.1, 38.9]",NaN,NaN,38.9,5,NaN,NaN,NaN,False,0.00,0.00
1160,0,128,8,"[nan, nan]","[51.3, 41.4, 41.6, 43.3, 54.1, 38.9]",NaN,NaN,38.9,5,NaN,NaN,NaN,False,0.00,0.00


In [116]:
class PPO(Trader):
    def __init__(self, gameData, disclosure, index, buyer, reinforcer):
        super().__init__(gameData, disclosure, index, buyer, reinforcer)
        self.learningRate = 0.0002
        self.gamma = 0.99
        self.numActions = 1
        self.depth = 20
        self.numStates = 8 + 6 * self.depth
        agentState = [0, -1, -1, -1, -1, -1, -1, -1]
        history = [-1, -1, -1, -1, -1, -1] * self.depth
        self.state = np.nan_to_num(np.array(agentState + history, dtype = np.float32), nan=-9)

        action_std = 0.02
        betas = [0.9, 0.990]
        max_episodes = 10000000
        max_timesteps = 100
        update_timesteps = 1
        action_std = 0.2
        K_epochs = 1
        eps_clip = 0.2
        gamma = 0.98
        lr = 0.0003
        seed = 123
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

        self.memory = Memory()
        self.ppo = PPOModel(self.numStates, self.numActions, action_std, lr, betas, gamma, K_epochs, eps_clip, restore=False, ckpt=None)
        self.done = False
        self.time_step = 0
        self.update_timestep = 1

    def observe(self):
        self.time_step += 1
        agentState = [self.periodStep, self.stepTokenValue, self.stepBid, self.stepAsk, self.stepTrades, self.stepProfits, self.periodTrades, self.periodProfits]
        history = self.df.iloc[-self.depth:][['currentBid', 'currentAsk', 'buy', 'sell', 'price', 'price']].values.reshape(-1,).tolist()
        if len(history) != (6 * self.depth):
            history = [-1] * (6 * self.depth)
        self.newState = np.nan_to_num(np.array(agentState + history, dtype = np.float32), nan=-9)
        if self.periodStep == self.numSteps:
            self.done = True
        self.memory.rewards.append(self.stepProfits)
        self.memory.is_terminals.append(self.done)
        self.state = self.newState

    def train(self):
        #if self.time_step % self.update_timestep == 0:
        self.ppo.update(self.memory)
        self.memory.clear_memory()
        self.time_step = 0
        
    def bid(self):
        self.stepBid = np.nan
        self.action = self.ppo.select_action(self.state, self.memory)
        if self.stepTokenValue > 0:
            frac = (np.clip(self.action.item(),-1,1)+1)/2
            self.stepBid = np.round(frac * 100,1)
        return self.stepBid
        
    def ask(self):
        self.stepAsk = np.nan
        self.action = self.ppo.select_action(self.state, self.memory)
        if self.stepTokenValue > 0:
            frac = self.action.item()
            self.stepAsk = np.round(frac * 100,1)
        return self.stepAsk

def updatePolicy(buyers, sellers):
    for buyer in buyers:
        if buyer.reinforcer == 1:
            buyer.train()
    for seller in sellers:
        if seller.reinforcer == 1:
            seller.train()

def updateStates(buyers, sellers):
    for buyer in buyers:
        if buyer.reinforcer == 1:
            buyer.observe()
    for seller in sellers:
        if seller.reinforcer == 1:
            seller.observe()

In [60]:
# This is a PPO algorithm for multi-dimension continuous action



import torch
import torch.nn as nn
from torch.distributions import MultivariateNormal
import gym
import numpy as np
import os
import argparse
from tensorboardX import SummaryWriter


class Memory:   # collected from old policy
    def __init__(self):
        self.states = []
        self.actions = []
        self.rewards = []
        self.is_terminals = []
        self.logprobs = []

    def clear_memory(self):
        del self.states[:]
        del self.actions[:]
        del self.rewards[:]
        del self.is_terminals[:]
        del self.logprobs[:]


class ActorCritic(nn.Module):
    def __init__(self, state_dim, action_dim, action_std):
        super(ActorCritic, self).__init__()
        global device
        self.actor = nn.Sequential(
            nn.Linear(state_dim, 64),
            nn.Tanh(),
            nn.Linear(64, 32),
            nn.Tanh(),
            nn.Linear(32, action_dim),
            nn.Tanh()
        )

        self.critic = nn.Sequential(
            nn.Linear(state_dim, 64),
            nn.Tanh(),
            nn.Linear(64, 32),
            nn.Tanh(),
            nn.Linear(32, 1)
        )

        self.action_var = torch.full((action_dim, ), action_std * action_std)    #(4, )

    def act(self, state, memory):       # state (1,24)
        action_mean = self.actor(state)                     # (1,4)
        cov_mat = torch.diag(self.action_var)   # (4,4)
        dist = MultivariateNormal(action_mean, cov_mat)
        action = dist.sample()                              # (1,4)
        action_logprob = dist.log_prob(action)

        memory.states.append(state)
        memory.actions.append(action.float())
        memory.logprobs.append(action_logprob)

        return action.detach().float()


    def evaluate(self, state, action):      # state (4000, 24); action (4000, 4)
        state_value = self.critic(state)    # (4000, 1)
        
        # to calculate action score(logprobs) and distribution entropy
        action_mean = self.actor(state)                     # (4000,4)
        action_var = self.action_var.expand_as(action_mean) # (4000,4)
        cov_mat = torch.diag_embed(action_var)   # (4000,4,4)

        dist = MultivariateNormal(action_mean, cov_mat)
        action = action.view(-1, 1) 
        action_logprobs = dist.log_prob(action)
        dist_entropy = dist.entropy()
        return action_logprobs, torch.squeeze(state_value), dist_entropy


class PPOModel:
    def __init__(self, state_dim, action_dim, action_std, lr, betas, gamma, K_epochs, eps_clip, restore=False, ckpt=None):
        self.lr = lr
        self.betas = betas
        self.gamma = gamma
        self.eps_clip = eps_clip
        self.K_epochs = K_epochs

        # current policy
        self.policy = ActorCritic(state_dim, action_dim, action_std)
        if restore:
            pretained_model = torch.load(ckpt, map_location=lambda storage, loc: storage)
            self.policy.load_state_dict(pretained_model)
        self.optimizer = torch.optim.Adam(self.policy.parameters(), lr=lr, betas=betas)
        
        # old policy: initialize old policy with current policy's parameter
        self.old_policy = ActorCritic(state_dim, action_dim, action_std)
        self.old_policy.load_state_dict(self.policy.state_dict())

        self.MSE_loss = nn.MSELoss()

    def select_action(self, state, memory):
        state = torch.FloatTensor(state.reshape(1, -1)).float()   # flatten the state
        #return self.old_policy.act(state, memory).cpu().numpy().flatten()
        return self.old_policy.act(state, memory).cpu().numpy().flatten().astype(float)

    
    def update(self, memory):
        # Monte Carlo estimation of rewards
        rewards = []
        discounted_reward = 0
        for reward, is_terminal in zip(reversed(memory.rewards), reversed(memory.is_terminals)):
            if is_terminal:
                discounted_reward = 0
            discounted_reward = reward + self.gamma * discounted_reward
            rewards.insert(0, discounted_reward)
        
        # Normalize rewards
        rewards = torch.tensor(rewards).float() 
        rewards = (rewards - rewards.mean()) / (rewards.std() + 1e-5)
        
        # Convert list to tensor
        old_states = torch.squeeze(torch.stack(memory.states)).detach()
        old_actions = torch.squeeze(torch.stack(memory.actions)).detach()
        old_logprobs = torch.squeeze(torch.stack(memory.logprobs)).detach()

        # Train policy for K epochs: sampling and updating
        for _ in range(self.K_epochs):
            # Evaluate old actions and values using current policy
            logprobs, state_values, dist_entropy = self.policy.evaluate(old_states, old_actions)

            # Importance ratio: p/q
            ratios = torch.exp(logprobs - old_logprobs.detach()).float()

            # Advantages
            advantages = rewards - state_values.detach()

            # Actor loss using Surrogate loss
            surr1 = ratios * advantages
            surr2 = torch.clamp(ratios, 1 - self.eps_clip, 1 + self.eps_clip) * advantages
            actor_loss = -torch.min(surr1, surr2)
    
            # Critic loss: critic loss - entropy

            critic_loss = 0.5 * self.MSE_loss(rewards, state_values) - 0.01 * dist_entropy
    
            # Total loss
            loss = actor_loss + critic_loss
    
            # Backward gradients
            self.optimizer.zero_grad()
            loss.mean().backward()
            self.optimizer.step()
    
        # Copy new weights to old_policy
        self.old_policy.load_state_dict(self.policy.state_dict())

def train(env_name, env, state_dim, action_dim, render, solved_reward,
    max_episodes, max_timesteps, update_timestep, action_std, K_epochs, eps_clip,
    gamma, lr, betas, ckpt_folder, restore, tb=False, print_interval=10, save_interval=100):

    ckpt = ckpt_folder+'/PPO_continuous_'+env_name+'.pth'
    if restore:
        print('Load checkpoint from {}'.format(ckpt))

    memory = Memory()

    ppo = PPO(state_dim, action_dim, action_std, lr, betas, gamma, K_epochs, eps_clip, restore=restore, ckpt=ckpt)

    running_reward, avg_length, time_step = 0, 0, 0

    # training loop
    for i_episode in range(1, max_episodes+1):
        state = env.reset()
        for t in range(max_timesteps):
            time_step += 1

            # Run old policy
            action = ppo.select_action(state, memory)

            state, reward, done, _ = env.step(action)

            memory.rewards.append(reward)
            memory.is_terminals.append(done)

            if time_step % update_timestep == 0:
                ppo.update(memory)
                memory.clear_memory()
                time_step = 0

            running_reward += reward

            if done:
                break
        avg_length += t


In [59]:
def generateAgents(gameData,buyerStrategies,sellerStrategies,disclosure):
    buyers, sellers = [], []
    for idx,i in enumerate(buyerStrategies):
        if i == 'TruthTeller':
            buyers.append(TruthTeller(gameData, disclosure, index=idx, buyer=1, reinforcer=0)) 
        if i == 'ZeroIntelligence':
            buyers.append(ZeroIntelligence(gameData, disclosure, index=idx, buyer=1, reinforcer=0)) 
        if i == 'REINFORCE':
            buyers.append(REINFORCE(gameData, disclosure, index=idx, buyer=1, reinforcer=1)) 
        if i == 'PPO':
            buyers.append(PPO(gameData, disclosure, index=idx, buyer=1, reinforcer=1)) 
        if i == 'SAC':
            buyers.append(SAC(gameData, disclosure, index=idx, buyer=1, reinforcer=1)) 
        if i == 'DQN':
            buyers.append(DQN(gameData, disclosure, index=idx, buyer=1, reinforcer=1)) 

    for idx,i in enumerate(sellerStrategies):
        if i == 'TruthTeller':
            sellers.append(TruthTeller(gameData, disclosure, index=idx, buyer=0, reinforcer=0)) 
        if i == 'ZeroIntelligence':
            sellers.append(ZeroIntelligence(gameData, disclosure, index=idx, buyer=0, reinforcer=0)) 
        if i == 'REINFORCE':
            sellers.append(REINFORCE(gameData, disclosure, index=idx, buyer=0, reinforcer=1)) 
        if i == 'PPO':
            sellers.append(PPO(gameData, disclosure, index=idx, buyer=0, reinforcer=1)) 
        if i == 'SAC':
            sellers.append(SAC(gameData, disclosure, index=idx, buyer=0, reinforcer=1)) 
        if i == 'DQN':
            sellers.append(DQN(gameData, disclosure, index=idx, buyer=0, reinforcer=1)) 
    return buyers, sellers

In [ ]:
import gym
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Normal
from torch.distributions import Beta

learning_rate  = 0.0003
gamma          = 0.9
lmbda          = 0.9
eps_clip       = 0.2
K_epoch        = 10
rollout_len    = 3
buffer_size    = 10
minibatch_size = 32

class PPO(nn.Module):
    def __init__(self, numStates):
        super(PPOModel, self).__init__()
        self.numStates = numStates
        self.data = []
        self.fc1   = nn.Linear(self.numStates,128)
        self.fc_mu = nn.Linear(128,1)
        self.fc_std  = nn.Linear(128,1)
        self.fc_v = nn.Linear(128,1)
        self.optimizer = optim.Adam(self.parameters(), lr=learning_rate)
        self.optimization_step = 0

    def pi(self, x, softmax_dim = 0):
        x = F.relu(self.fc1(x))
        mu = torch.sigmoid(self.fc_mu(x))
        std = 0.4*F.sigmoid(self.fc_std(x))
        return mu, std
    
    def v(self, x):
        x = F.relu(self.fc1(x))
        v = self.fc_v(x)
        return v
      
    def put_data(self, transition):
        self.data.append(transition)
        
    def make_batch(self):
        s_batch, a_batch, r_batch, s_prime_batch, prob_a_batch, done_batch = [], [], [], [], [], []
        data = []
        for j in range(buffer_size):
            for i in range(minibatch_size):
                rollout = self.data.pop()
                s_lst, a_lst, r_lst, s_prime_lst, prob_a_lst, done_lst = [], [], [], [], [], []
                for transition in rollout:
                    s, a, r, s_prime, prob_a, done = transition      
                    s_lst.append(s)
                    a_lst.append([a])
                    r_lst.append([r])
                    s_prime_lst.append(s_prime)
                    prob_a_lst.append([prob_a])
                    done_mask = 0 if done else 1
                    done_lst.append([done_mask])
                s_batch.append(s_lst)
                a_batch.append(a_lst)
                r_batch.append(r_lst)
                s_prime_batch.append(s_prime_lst)
                prob_a_batch.append(prob_a_lst)
                done_batch.append(done_lst)
            mini_batch = torch.tensor(s_batch, dtype=torch.float), torch.tensor(a_batch, dtype=torch.float), \
                          torch.tensor(r_batch, dtype=torch.float), torch.tensor(s_prime_batch, dtype=torch.float), \
                          torch.tensor(done_batch, dtype=torch.float), torch.tensor(prob_a_batch, dtype=torch.float)
            data.append(mini_batch)
        return data

    def calc_advantage(self, data):
        data_with_adv = []
        for mini_batch in data:
            s, a, r, s_prime, done_mask, old_log_prob = mini_batch
            with torch.no_grad():
                td_target = r + gamma * self.v(s_prime) * done_mask
                delta = td_target - self.v(s)
            delta = delta.numpy()
            advantage_lst = []
            advantage = 0.0
            for delta_t in delta[::-1]:
                advantage = gamma * lmbda * advantage + delta_t[0]
                advantage_lst.append([advantage])
            advantage_lst.reverse()
            advantage = torch.tensor(advantage_lst, dtype=torch.float)
            data_with_adv.append((s, a, r, s_prime, done_mask, old_log_prob, td_target, advantage))
        return data_with_adv

    def train_net(self):
        if len(self.data) == minibatch_size * buffer_size:
            data = self.make_batch()
            data = self.calc_advantage(data)
            for i in range(K_epoch):
                for mini_batch in data:
                    s, a, r, s_prime, done_mask, old_log_prob, td_target, advantage = mini_batch
                    mu, std = self.pi(s, softmax_dim=1)
                    dist = Normal(mu, std)
                    log_prob = dist.log_prob(a)
                    ratio = torch.exp(log_prob - old_log_prob)  # a/b == exp(log(a)-log(b))
                    surr1 = ratio * advantage
                    surr2 = torch.clamp(ratio, 1-eps_clip, 1+eps_clip) * advantage
                    loss = -torch.min(surr1, surr2) + F.smooth_l1_loss(self.v(s) , td_target)
                    self.optimizer.zero_grad()
                    loss.mean().backward()
                    nn.utils.clip_grad_norm_(self.parameters(), 1.0)
                    self.optimizer.step()
                    self.optimization_step += 1
